In [52]:
import requests
import pandas as pd
import os
import wget

In [53]:
root_folder = 'dados/'

folder_renach = 'renach/'
folder_infracoes = 'infracoes/'

#### Lendo dados dos arquivos do RENACH

In [54]:
df_result = pd.DataFrame()

for file in os.listdir(root_folder + folder_renach):
    df = pd.read_csv(root_folder + folder_renach + file, encoding='utf_16', decimal=',', thousands='.')

    file = file.replace('-', '_')  
    df['ano_mes'] = int(file.split('_')[2] + file.split('_')[3].split('.')[0])
    df_result = pd.concat([df_result, df], axis='index')

#### Adiquirindo a relação de nome/sigla das UFs

In [55]:
response = requests.get('http://servicodados.ibge.gov.br/api/v1/localidades/estados?orderBy=id').json()

In [56]:
ufs = {}
for uf in response: ufs[uf['nome']] = uf['sigla']

In [57]:
df_result['uf'] = [ufs[nome_uf] for nome_uf in df_result['UF Habilitação Atual']]
df_result['categoria_cnh'] = df_result['Categoria'].str.split('-', expand=True)[0]

In [58]:
df_result = df_result[['uf', 'Sexo', 'Faixa Etária', 'categoria_cnh', 'Qt. Condutor Histórico', 'ano_mes']]

quantidade_condutores = df_result.rename(columns={
    'Sexo':'sexo',
    'Faixa Etária':'faixa_etaria',
    'Qt. Condutor Histórico':'qtd_condutores'
})

In [59]:
quantidade_condutores

,uf,sexo,faixa_etaria,categoria_cnh,qtd_condutores,ano_mes
0,DF,Masculino,18 a 21,A,105,202202
1,DF,Masculino,18 a 21,AB,9881,202202
2,DF,Masculino,18 a 21,AC,6,202202
3,DF,Masculino,18 a 21,AD,29,202202
4,DF,Masculino,18 a 21,B,15079,202202
...,...,...,...,...,...,...
4607,SC,Feminino,91 a 100,A,2,201801
4608,SC,Feminino,91 a 100,AB,5,201801
4609,SC,Feminino,91 a 100,B,672,201801
4610,SC,Feminino,91 a 100,D,1,201801


#### Adiquirindo dados sobre os tipos de infrações

In [60]:
df_infracoes = pd.read_excel(root_folder + folder_infracoes + 'tabela-codigo-infracoes-renainf-xlsx.xlsx', engine='openpyxl')

In [61]:
df_infracoes['id'] = df_infracoes.index + 1
df_infracoes = df_infracoes[['id', 'Código da Infração', 'Descrição da Infração', 'Gravidade', 'Órgão Competente']]

infracoes = df_infracoes.rename(columns={
    'Código da Infração':'codigo_infracao',
    'Descrição da Infração':'descricao_infracao',
    'Gravidade':'gravidade',
    'Órgão Competente':'orgao_competente'
})

In [62]:
infracoes

,id,codigo_infracao,descricao_infracao,gravidade,orgao_competente
0,1,5002,"Multa, por não identificação do condutor infra...",---,EST/MUNIC/RODOV
1,2,5010,Dirigir veículo sem possuir CNH ou Permissão p...,7 - Gravíss 3X,ESTADUAL/RODOV
2,3,5029,Dirigir veículo com CNH ou PPD cassada,7 - Gravíss 5X,ESTADUAL/RODOV
3,4,5029,Dirigir veículo com CNH ou PPD com suspensão d...,7 - Gravíss 5X,ESTADUAL/RODOV
4,5,5037,Dirigir veículo com CNH de categoria diferente...,7 - Gravíss 3X,ESTADUAL/RODOV
...,...,...,...,...,...
253,254,7579,Cond que se recusar a se submeter a qq dos pro...,Gravíss 10X,ESTADUAL/RODOV
254,255,7587,Transitar na faixa ou via exclusiva regulam. p...,7-Gravíss,MUNICIPAL/RODOV
255,256,7595,Dirigir veículo realizando cobrança de tarifa ...,Média,ESTAD/MUNIC/RODOV
256,257,7609,Organizar as condutas previstas no caput do ar...,Gravíss 100X,MUNICIPAL/RODOV


#### Adiquirindo dados do RENAEST

In [63]:
df_acidentes = pd.read_csv('dados/renaest/Acidentes_DadosAbertos_20230912.csv', sep=';')

/tmp/ipykernel_66276/1315882353.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_acidentes = pd.read_csv('dados/renaest/Acidentes_DadosAbertos_20230912.csv', sep=';')


In [64]:
df_acidentes.columns

Index(['num_acidente', 'chv_localidade', 'data_acidente', 'uf_acidente',
       'ano_acidente', 'mes_acidente', 'mes_ano_acidente', 'codigo_ibge',
       'dia_semana', 'fase_dia', 'tp_acidente', 'cond_meteorologica',
       'end_acidente', 'num_end_acidente', 'cep_acidente', 'bairro_acidente',
       'km_via_acidente', 'latitude_acidente', 'longitude_acidente',
       'hora_acidente', 'tp_rodovia', 'cond_pista', 'tp_cruzamento',
       'tp_pavimento', 'tp_curva', 'lim_velocidade', 'tp_pista',
       'ind_guardrail', 'ind_cantcentral', 'ind_acostamento', 'qtde_acidente',
       'qtde_acid_com_obitos', 'qtde_envolvidos', 'qtde_feridosilesos',
       'qtde_obitos'],
      dtype='object')

In [65]:
df_acidentes.head()

,num_acidente,chv_localidade,data_acidente,uf_acidente,ano_acidente,mes_acidente,mes_ano_acidente,codigo_ibge,dia_semana,fase_dia,...,lim_velocidade,tp_pista,ind_guardrail,ind_cantcentral,ind_acostamento,qtde_acidente,qtde_acid_com_obitos,qtde_envolvidos,qtde_feridosilesos,qtde_obitos
0,2448623,AC1200401201801,2018-01-24,AC,2018,1,12018,1200401,QUARTA-FEIRA,MANHA,...,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,1,0,3,3,0
1,3477494,AC1200385201801,2018-01-06,AC,2018,1,12018,1200385,SABADO,TARDE,...,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,1,0,1,1,0
2,2939219,AC1200401201801,2018-01-04,AC,2018,1,12018,1200401,QUINTA-FEIRA,MANHA,...,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,1,0,2,2,0
3,17595,AC1200203201801,2018-01-22,AC,2018,1,12018,1200203,SEGUNDA-FEIRA,NOITE,...,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,1,0,2,2,0
4,58010,AC1200401201801,2018-01-21,AC,2018,1,12018,1200401,DOMINGO,MANHA,...,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,NAO INFORMADO,1,0,2,2,0


In [66]:
df_localidade = pd.read_csv('dados/renaest/Localidade_DadosAbertos_20230912.csv', sep=';')

In [70]:
df_localidade.columns

Index(['chv_localidade', 'ano_referencia', 'mes_referencia',
       'mes_ano_referencia', 'regiao', 'uf', 'codigo_ibge', 'municipio',
       'regiao_metropolitana', 'qtde_habitantes', 'frota_total',
       'frota_circulante'],
      dtype='object')

In [51]:
df_localidade[(df_localidade['uf'] == 'AC') & (df_localidade['ano_referencia'] == 2018)]

,chv_localidade,ano_referencia,mes_referencia,mes_ano_referencia,regiao,uf,codigo_ibge,municipio,regiao_metropolitana,qtde_habitantes,frota_total,frota_circulante
0,AC1200708201801,2018,1,12018,NORTE,AC,1200708,XAPURI,nao,18174,3956,2824
1,AC1200807201801,2018,1,12018,NORTE,AC,1200807,PORTO ACRE,nao,17459,3764,2697
2,AC1200351201801,2018,1,12018,NORTE,AC,1200351,MARECHAL THAUMATURGO,nao,17897,313,206
3,AC1200328201801,2018,1,12018,NORTE,AC,1200328,JORDAO,nao,7858,101,58
4,AC1200138201801,2018,1,12018,NORTE,AC,1200138,BUJARI,nao,9664,2168,1587
...,...,...,...,...,...,...,...,...,...,...,...,...
271,AC1200393201812,2018,12,122018,NORTE,AC,1200393,PORTO WALTER,nao,11720,310,220
272,AC1200203201812,2018,12,122018,NORTE,AC,1200203,CRUZEIRO DO SUL,nao,87673,30387,21920
273,AC1200385201812,2018,12,122018,NORTE,AC,1200385,PLACIDO DE CASTRO,nao,19565,4889,3599
274,AC1200500201812,2018,12,122018,NORTE,AC,1200500,SENA MADUREIRA,nao,45177,8549,6788


#### Adiquirindo os dados de quantidades de infrações

In [89]:
os.chdir(root_folder + folder_infracoes)

for ano in [2019, 2020, 2021, 2022]:
    for mes in range(1, 13):    
        
        try:
            data = f'{ano}_{mes:02d}'
            url = f'https://www.gov.br/transportes/pt-br/assuntos/transito/arquivos-senatran/estatisticas/renainf/csv/{data}_infracoes_com_np.csv'
            
            wget.download(url)
        except:
            data = f'{ano}-{mes:02d}'
            url = f'https://www.gov.br/transportes/pt-br/assuntos/transito/arquivos-senatran/estatisticas/renainf/csv/{data}_infracoes_com_np.csv'

            wget.download(url)

os.chdir('../..')            

In [90]:
df_result = pd.DataFrame()

for file in os.listdir(root_folder + folder_infracoes):
    df = pd.read_csv(root_folder + folder_infracoes + file, sep=';', encoding='ISO-8859-1')

    file = file.replace('-', '_')  
    print(file)
    df['ano_mes'] = int(file.split('_')[0] + file.split('_')[1])
    df_result = pd.concat([df_result, df], axis='index')

2022_12_infracoes_com_np.csv
2022_02_infracoes_com_np.csv
2022_03_infracoes_com_np.csv
2020_07_infracoes_com_np.csv
2022_06_infracoes_com_np.csv
2021_09_infracoes_com_np.csv
2021_02_infracoes_com_np.csv
2020_02_infracoes_com_np.csv
2022_01_infracoes_com_np.csv
2021_06_infracoes_com_np.csv
2019_03_infracoes_com_np.csv
2020_06_infracoes_com_np.csv
2021_01_infracoes_com_np.csv


ParserError: Error tokenizing data. C error: Expected 1 fields in line 9, saw 2


In [86]:
df = pd.read_excel('/home/lucas/Downloads/2022_09_infracoes_com_np.xlsx', engine='openpyxl')

/home/lucas/.miniconda3/envs/siad/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [88]:
df

,UF Jurisdição Veículo (Desc),ACRE,ALAGOAS,AMAZONAS,AMAPA,BAHIA,CEARA,DISTRITO FEDERAL,ESPIRITO SANTO,GOIAS,...,PIAUI,RIO DE JANEIRO,RIO GRANDE DO NORTE,RIO GRANDE DO SUL,RONDONIA,RORAIMA,SANTA CATARINA,SAO PAULO,SERGIPE,TOCANTINS
0,Infração,Qt Infração c/ NP,Qt Infração c/ NP,Qt Infração c/ NP,Qt Infração c/ NP,Qt Infração c/ NP,Qt Infração c/ NP,Qt Infração c/ NP,Qt Infração c/ NP,Qt Infração c/ NP,...,Qt Infração c/ NP,Qt Infração c/ NP,Qt Infração c/ NP,Qt Infração c/ NP,Qt Infração c/ NP,Qt Infração c/ NP,Qt Infração c/ NP,Qt Infração c/ NP,Qt Infração c/ NP,Qt Infração c/ NP
1,5002,102,55,186,20,1046,182,758,1178,4438,...,111,2542,95,10017,324,21,19607,173528,126,231
2,5010,400,1358,162,482,2572,1195,2969,1663,656,...,442,1754,745,2189,1574,75,3074,3367,538,199
3,5029,3,8,NaN,1,12,9,206,158,20,...,NaN,81,3,234,9,4,490,739,5,2
4,5037,20,54,10,30,117,89,219,78,61,...,23,135,34,366,36,2,123,408,26,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,7625,191,98,176,23,476,454,3541,258,222,...,250,1274,157,318,35,261,938,3117,218,32
242,7633,639,880,683,145,5367,1606,35331,1739,2639,...,1223,3918,2159,2410,587,133,7712,41244,1541,195
243,7684,7,139,4,22,422,5749,673,162,662,...,45,98,66,563,27,5,2525,14800,38,12
244,7714,2,13,2,1,22,35,26,16,45,...,25,8,7,19,7,6,69,603,6,1


In [19]:
localidade_url = 'http://servicodados.ibge.gov.br/api/v1/localidades/municipios'

response = requests.get(url=localidade_url).json()